In [120]:
%run gen_cv.py


Building Trie..., from /home/joekaojoekao/github/dict.txt.big
DEBUG:jieba:Building Trie..., from /home/joekaojoekao/github/dict.txt.big
loading model from cache /tmp/jieba.user.8815186977980235228.cache
DEBUG:jieba:loading model from cache /tmp/jieba.user.8815186977980235228.cache
loading model cost 1.74812197685 seconds.
DEBUG:jieba:loading model cost 1.74812197685 seconds.
Trie has been built succesfully.
DEBUG:jieba:Trie has been built succesfully.


read source: ./visualized/select_pushes1000_1.txt
Pushes filtering (keep only pushes larger than 4 words)
After filtering / Origin push num:  999 / 999
Users filtering (remove users who post less than 3 pushes)
After filtering / Previous user num:  718 / 999
write user list: ./temp/select_pushes1000_1_userlist.txt
write push data ready to do cross_validation: ./temp/select_pushes1000_1_cvlist.txt
2


In [129]:
import os
dir_path = os.getcwd() + '/'
with open(dir_path + 'config.txt', 'rb') as f_conf:
    config = json.load(f_conf)
    f_conf.close()
    
sample_file = config["sample_file"] #first input
temp_path = config["temp_path"] 
result_dir = config["result_dir"]

## just pick some user here from temp folder
with open(temp_path + os.path.splitext(sample_file)[0] + '_userlist.txt', 'rb') as f_temp:
    print 'read user from ', temp_path + os.path.splitext(sample_file)[0] + '_userlist.txt'
    user_list = json.load(f_temp)
    f_temp.close()

with open(temp_path + os.path.splitext(sample_file)[0] + '_cvlist.txt', 'rb') as f_temp:
    print 'read push from', temp_path + os.path.splitext(sample_file)[0] + '_cvlist.txt'
    temp_json = json.load(f_temp)
    f_temp.close()
    

dict_train, dict_test = temp_json # train/test push list for each user

print len(dict_train['heavenkghs'])
print len(dict_test['heavenkghs'])

read user from  ./temp/select_pushes1000_1_userlist.txt
read push from ./temp/select_pushes1000_1_cvlist.txt
4
4


In [136]:
%%pypy 
## using pypy magic needs a temp file for output 
import json
import numpypy as np # while at sweslos' centos6.x
import time

def count_dict(sample_dict_jieba):
    from collections import Counter
    from collections import OrderedDict
    import itertools
    ptt_pushes_freq_bypush = {}
    for uid, push_list in sample_dict_jieba.iteritems():
        push = push_list[0] #raw push
        a = zip(*push_list)[1] #jieba ones
        push_gram = list(itertools.chain(*a))
        count = Counter(push_gram)
        temp = []
        for w, c in count.most_common():
            temp.append((w, c))
        id_count = Counter(dict(temp))
        ptt_pushes_freq_bypush[uid] = dict(id_count)

    return ptt_pushes_freq_bypush

def weighted_jaccard(l1, l2):
    if len(l1) != len(l2):
        return -1
    num = 0
    den = 0
    for i in xrange(len(l1)):
        num += np.minimum(l1[i], l2[i])
        den += np.maximum(l1[i], l2[i])
    wj = np.divide(np.float64(num), den)

    return wj

import os
dir_path = os.getcwd() + '/'
with open(dir_path + 'config.txt', 'rb') as f_conf:
    config = json.load(f_conf)
    f_conf.close()
    
sample_file = config["sample_file"] #first input
temp_path = config["temp_path"] 
result_dir = config["result_dir"]

## just pick some user here from temp folder
with open(temp_path + os.path.splitext(sample_file)[0] + '_userlist.txt', 'rb') as f_temp:
    print 'read user from ', temp_path + os.path.splitext(sample_file)[0] + '_userlist.txt'
    user_list = json.load(f_temp)
    f_temp.close()

with open(temp_path + os.path.splitext(sample_file)[0] + '_cvlist.txt', 'rb') as f_temp:
    print 'read push from', temp_path + os.path.splitext(sample_file)[0] + '_cvlist.txt'
    temp_json = json.load(f_temp)
    f_temp.close()
    

dict_train, dict_test = temp_json # train/test push list for each user

dict_train_count = count_dict(dict_train) #count the freq
dict_test_count = count_dict(dict_test) #count the freq for ans(test)

from collections import Counter
count_all = Counter()
for v in dict_train_count.values():
    count_all += Counter(v)

from collections import OrderedDict
sorted_tuple_list_train_count = OrderedDict(sorted(dict(count_all).items(), key=lambda t: t[1], reverse=True))
## global term count using train data

# with open(temp_path + os.path.splitext(sample_file)[0] + '_train_count.txt', 'wb') as f_temp:
#     f_temp.write(json.dumps(dict_train_count, indent=2, ensure_ascii=True).encode('utf-8'))
#     f_temp.close()

################

##test function

#################Feature Extraction##########################

import pttfeat

selected_cw = pttfeat.CW(dict_train_count)
#print selected_cw

selected_aw = pttfeat.AW(dict_train_count)
#print selected_aw

def user_sen_distrib(user_sl):
    "return distribution of user sentence length"
    LOW = 5
    HIGH = 20
    sl_stat = {}
    for k in user_sl.keys():
        user_cut = []
        temp = []
        for push_sl in user_sl[k]:
            push_cut = push_sl['cut']
            for s in push_cut:
                if s <= LOW:
                    temp.append(0)
                elif s > LOW and s <= HIGH:
                    temp.append(1)
                elif s > HIGH:
                    temp.append(2) 
        temp_percent = [float(x) / len(temp) for x in [temp.count(0), temp.count(1), temp.count(2)]]
        #print k,temp_percent
        sl_stat[k] = temp_percent
    return sl_stat

user_sl_train = pttfeat.SL(dict_train)
sl_stat_train = user_sen_distrib(user_sl_train)

user_sl_test = pttfeat.SL(dict_test)
sl_stat_test = user_sen_distrib(user_sl_test)

    
# with open('./feature_pool/' + 'CW.txt', 'wb') as f_feature:
#     f_feature.write(json.dumps(selected_cw, indent=2, ensure_ascii=True).encode('utf-8'))
#     f_feature.close()

###############VEC TIME##########################################

## 1. gen sw vec
# import itertools
# general_vec = {}
# for uid in user_list: # for each user id
#     user_len = sum([len(x) for x in dict_train_count[uid]]) #total word freq
#     if user_len > 0:
#         vec = [dict_train_count[uid].get(w, 0) for w in selected_cw]
#         g_vec = [float(x) / user_len for x in vec]
#         general_vec[uid] = g_vec


# general_vec_ans = {}
# for uid in user_list: # for each user id
#     user_len = sum([len(x) for x in dict_test_count[uid]]) #total word freq
#     #print sum(v.values())
#     if user_len > 0:
#         vec = [dict_test_count[uid].get(w, 0) for w in selected_cw]
#         g_vec = [float(x) / user_len for x in vec]
#         general_vec_ans[uid] = g_vec

# print len(general_vec[user_list[0]])
# print len(general_vec_ans[user_list[0]])



## 2. gen aw vec
import itertools
general_vec = {}
for uid in user_list: # for each user id
    user_len = sum([len(x) for x in dict_train_count[uid]]) #total word freq
    if user_len > 0:
        vec = [dict_train_count[uid].get(w, 0) for w in selected_aw]
        g_vec = [float(x) / user_len for x in vec]
        general_vec[uid] = g_vec


general_vec_ans = {}
for uid in user_list: # for each user id
    user_len = sum([len(x) for x in dict_test_count[uid]]) #total word freq
    #print sum(v.values())
    if user_len > 0:
        vec = [dict_test_count[uid].get(w, 0) for w in selected_aw]
        g_vec = [float(x) / user_len for x in vec]
        general_vec_ans[uid] = g_vec

print user_list[0]
print general_vec[user_list[0]]
print general_vec_ans[user_list[0]]


## 3. gen sl vec
# import itertools
# general_vec = {}
# for uid in user_list: # for each user id
#     #user_len = sum([len(x) for x in dict_train_count[uid]]) #total word freq
#     #if user_len > 0:
#         #vec = [dict_train_count[uid].get(w, 0) for w in selected_cw]
#         #g_vec = [float(x) / user_len for x in vec]
#     general_vec[uid] = sl_stat_train[uid]


# general_vec_ans = {}
# for uid in user_list: # for each user id
#     #user_len = sum([len(x) for x in dict_test_count[uid]]) #total word freq
#     #print sum(v.values())
#     #if user_len > 0:
#         #vec = [dict_test_count[uid].get(w, 0) for w in selected_cw]
#         #g_vec = [float(x) / user_len for x in vec]
#     general_vec_ans[uid] = sl_stat_test[uid]

    
# print len(general_vec[user_list[0]])
# print len(general_vec_ans[user_list[0]])





t_start = time.time()

user_size = len(user_list)
print user_size

sim_list = np.array(np.arange(user_size*user_size))
sim_list = sim_list.astype(float)
print '...'
idx = 0
for i in xrange(user_size):
    for j in xrange(user_size):
        #wj_sw = weighted_jaccard(general_vec[user_list[i]], general_vec_ans[user_list[j]])
        #sim_list = np.vstack((sim_list, np.array((round(float(i),1), round(float(j),1), wj_sw))))

        sim_list[idx] = weighted_jaccard(general_vec[user_list[i]], general_vec_ans[user_list[j]])
        idx += 1
sim_list = sim_list[:idx]
t_stop = time.time()
print sim_list
print 'sim compare time = ', t_stop - t_start, 's'

user_sim_list = []
idx = 0
for i in xrange(user_size):
    for j in xrange(user_size):
        sim = sim_list[idx]
        idx += 1
        user_sim_list.append((user_list[int(i)], user_list[int(j)] + 'ANS', sim))
#name = os.path.splitext(sample_file)[0]
with open(result_dir + os.path.splitext(sample_file)[0] + '_cv.txt', 'wb') as fout:
    resultpath = result_dir + os.path.splitext(sample_file)[0] + '_cv.txt'
    print 'write to ', resultpath 
    for user, ans, sim in user_sim_list:
        line = user + ',' + ans + ',' + str(sim) +'\n'
        fout.write(line.encode('utf-8'))
    fout.close()


read user from  ./temp/select_pushes1000_1_userlist.txt
read push from ./temp/select_pushes1000_1_cvlist.txt
total word 9507
stop word 249
num_word: 1 	132
num_word: 2 	107
num_word: 3 	7
246
total word 9507
rare word 6073
num_word: 1 	447
num_word: 2 	4048
num_word: 3 	991
num_word: 4 	394
num_word: 5 	50
num_word: 6 	34
5964
3
3
718
...
[ 0.          0.          1.         ...,  0.          0.33333333  1.        ]
sim compare time =  0.209052085876 s
write to  ./cv_result/select_pushes1000_1_cv.txt


In [ ]:
j_len_code = []
for j in j_len:
    temp = []
    for s in j:
        if s < LOW:
            temp.append(0)
        elif s >= LOW and s < HIGH:
            temp.append(1)
        elif s >= HIGH:
            temp.append(2)
    j_len_code.append(temp)


In [46]:
%%pypy
import pttfeat
print pttfeat.cut_sentence("你把他們當同胞,但他們第一個出賣的就是你時。不覺醒很難")
a = pttfeat.cut_sentence("你把他們當同胞,但他們第一個出賣的就是你時。不覺醒很難")
for aa in a:
    print aa.encode('utf-8')

[u'\u4f60\u628a\u4ed6\u5011\u7576\u540c\u80de', u'\u4f46\u4ed6\u5011\u7b2c\u4e00\u500b\u51fa\u8ce3\u7684\u5c31\u662f\u4f60\u6642', u'\u4e0d\u89ba\u9192\u5f88\u96e3']
你把他們當同胞
但他們第一個出賣的就是你時
不覺醒很難


In [100]:
PUNCTUATION_TABLE = [
    # start,  stop
    (0x2000, 0x206f),  # General Punctuation
    (0x3000, 0x303f),  # CJK Symbols and Punctuation
    (0xff00, 0xffef),  # Halfwidth and Fullwidth Forms
    (0x00, 0x2f),  # ascii special charactors
    # (0x30, 0x39), # number 0~9
    (0x3a, 0x40),  # ascii special charactors
    (0x5b, 0x60),  # ascii special charactors
    (0x7b, 0xff),  # ascii special charactors
]
PUNCTUATION_RANGE = reduce(lambda x, y: x.union(y), [
                           range(start, stop + 1) for start, stop in PUNCTUATION_TABLE], set())



def isPUNC(uchar):
    if ord(uchar) in PUNCTUATION_RANGE:
        return True
    else:
        return False
text = "Zhon这个Python库提sdf供了常用汉字常sdfs量，如CJK字符和偏旁,.,.,..<><>，中文标点，，，，拼音，和汉字正则表达式（如找到文本中的繁。体字）。："
#title = re.sub('[%s]' % zhon.hanzi.punctuation, " ", text)
#title = re.sub('[%s]' % sep_list, " ", title)
#print title



def cut_sentence(push): ##放入原始文章路徑, 增加斷詞的list
    #text = codecs.open(text_path,"r","utf-8")   #開檔
    PUNCTUATION_TABLE = [
    # start,  stop
    (0x2000, 0x206f),  # General Punctuation
    (0x3000, 0x303f),  # CJK Symbols and Punctuation
    (0xff00, 0xffef),  # Halfwidth and Fullwidth Forms
    (0x00, 0x2f),  # ascii special charactors
    # (0x30, 0x39), # number 0~9
    (0x3a, 0x40),  # ascii special charactors
    (0x5b, 0x60),  # ascii special charactors
    (0x7b, 0xff),  # ascii special charactors
    ]
    PUNCTUATION_RANGE = reduce(lambda x, y: x.union(y), [
                           range(start, stop + 1) for start, stop in PUNCTUATION_TABLE], set())

    push = push.decode('utf-8')
    sent = ""
    punt_list = ',.!?:;~，。！？：；～'.decode('utf-8')
    cut_list = "[。，,！……!《》<>\"':：？/?、/|“”‘’；]{}（）{}【】()｛｝（）：？！。，;、~——+％%`:“”＂'‘\n\r".decode('utf-8')
    import zhon.hanzi # external lib for chinese punc 
    sep_list = list(set(punt_list + cut_list)) #customized punc list
    sent_list = []
       
    for word in push:
        if ord(word) not in PUNCTUATION_RANGE: #如果文字不是標點符號，就把字加到句子中
            sent += word
            #print sentence
        #use re.sub |
        #if word not in zhon.hanzi.punctuation:
            #sent += word
        #if ord(word) not in PUNCTUATION_RANGE:
            #sent += word
        else:
            sentence = sent.strip()
            if len(sentence) != 0:
                sent_list.append(sentence.strip()) #如果遇到標點符號，把句子加到 text list中
            sent = ""
            #print textList

    return sent_list#傳回一個文字陣列
for s in cut_sentence(text):
    print s


Zhon这个Python库提sdf供了常用汉字常sdfs量
如CJK字符和偏旁
中文标点
拼音
和汉字正则表达式
如找到文本中的繁
体字


In [101]:
def SL(dict_train):
    print dict_train[0]

In [48]:
def FindTok(cutlist, char):
    if char in cutlist:
        return True
    else:
        return False
     
def Cut(cutlist,lines):   
    l = []   
    line = []   
        
    for i in lines:   
        if FindTok(cutlist,i):
            line.append(i)
            l.append(''.join(line))
            #l.append(i)
            line = []   
        else:   
            line.append(i)   
    return l  
 
cutlist ="[。，,！……!《》<>\"':：？\?、\|“”‘’；]{}（）{}【】()｛｝（）：？！。，;、~——+％%`:“”＂'‘\n\r".decode('utf-8')


In [56]:


# def cut_sentence_new(words):
#     # words = (words).decode('utf8')
#     start = 0
#     i = 0
#     sents = []
#     punt_list = ',.!?:;~，。！？：；～'.decode('utf8')
#     cutlist = "[。，,！……!《》<>\"':：？/?、/|“”‘’；]{}（）{}【】()｛｝（）：？！。，;、~——+％%`:“”＂'‘\n\r".decode('utf-8')
#     sep_list = list(set(punt_list + cut_list))
#     for word in words:
#         if word in sep_list and token not in sep_list: #检查标点符号下一个字符是否还是标点
#             sents.append(words[start:i+1])
#             start = i+1
#             i += 1
#         else:
#             i += 1
#             token = list(words[start:i+2]).pop() # 取下一个字符
#     if start < len(words):
#         sents.append(words[start:])
#     return sents

with open(temp_path + os.path.splitext(sample_file)[0] + '_cvlist_new.txt', 'rb') as f_temp:
    temp_json = json.load(f_temp)
    f_temp.close()
#print temp
sample_dict_jieba = temp_json[0]
import itertools
ptt_pushes_freq_bypush = {}
j = {}
c = 0
for uid, push_list in sample_dict_jieba.iteritems():
    #push = push_list[0] #raw push
    pushes = zip(*push_list)[0] #jieba ones
    #print a
    #print cut_sentence_new(a)
    print pushes
    for push in pushes:
        l = Cut(list(cutlist),list(push))
        print l
#     j[uid] = []
    
#     for line in l:
#         if line.strip() != "":
#             li = line.strip().split()
#             for sent in li:
#                 print sent
#         #j[uid].append()
    c+=1
    if c == 10:
        break
#print json.dumps(j)

# for lines in file("t.txt"):    
#     l = Cut(list(cutlist),list(lines.decode('gbk')))     
#     for line in l:  
#        if line.strip() !="":      
#             li = line.strip().split()   
#             for sentence in li:
#                 print sentence

(u'Push!', u'XDDDDDDDD', u'\u597d\u96e3\u904e \u4e0d\u8981\u6253\u4ed6\u5011', u'XDDDD')
[u'Push!']
[]
[]
[]
(u'\u70ba\u4ec0\u9ebc\u8aaa\u5be6\u8a71\u7684\u4eba\u6703\u88ab\u4e00\u7fa4\u75c5\u755c\u5653?', u'\u70ba\u4ec0\u9ebc\u6c34\u6876?', u'\u4e2d\u56fd\u4eba\u4f86\u9019\u9178\u7684\u771f\u6c92\u4eba\u6027,\u5168\u4e0d\u5f97\u597d\u6b7b', u'\u90fd\u6709\u4eba\u88ab\u8b66\u5bdf\u6253\u5230\u6027\u547d\u5782\u5371\u4e86,\u90a3\u4e9b\u4e2d\u56fd\u4eba\u548c\u9ee8\u5de5\u9084\u5728\u9178,\u6c92\u826f\u5fc3', u'\u52a0\u6cb9! \u5e78\u82e6\u4e86,\u795d\u90a3\u4e9b\u4f86\u5653\u7684\u4e2d\u56fd\u4eba\u548c\u9ee8\u5de5\u4e0d\u5f97\u597d\u6b7b', u'\u99ac\u82f1\u4e5d\u7684\u7684\u6b77\u53f2\u5b9a\u4f4d\u5c31\u662f"\u7368\u88c1"', u'\u63a8! \u570b\u6c11\u9ee8\u4e00\u76f4\u4ee5\u4f86\u548c\u9ed1\u9053\u90fd\u662f\u540c\u4e00\u5925', u'\u9019\u5c31\u662f\u9ee8\u5de5\u554a!')
[u'\u70ba\u4ec0\u9ebc\u8aaa\u5be6\u8a71\u7684\u4eba\u6703\u88ab\u4e00\u7fa4\u75c5\u755c\u5653?']
[u'\u70ba\u4ec0\u9ebc\u6c3

In [22]:
punt_list = ',.!?:;~，。！？：；～'.decode('utf8')
cut_list = "[。，,！……!《》<>\"':：？/?、/|“”‘’；]{}（）{}【】()｛｝（）：？！。，;、~——+％%`:“”＂'‘\n\r".decode('utf-8')
print len(punt_list)
print len(cut_list)
sep_list = list(set(punt_list + cut_list))
print len(sep_list)
print sep_list

14
64
48
[u'\uff01', u'\u3001', u'\u3002', u'\uff05', u'\uff09', u'\uff08', u'\u300b', u'\u300a', u'\r', u'\uff0c', u'\u3011', u'\u3010', u'\uff02', u'\u2014', u'\u2019', u'\u2018', u'\uff1b', u'\uff1a', u'\u201d', u'\u201c', u'\uff1f', u'!', u'"', u'%', u"'", u'\u2026', u')', u'(', u'+', u',', u'/', u'.', u';', u':', u'<', u'?', u'>', u'\n', u'\uff5b', u'[', u']', u'\uff5e', u'`', u'\uff5d', u'{', u'}', u'|', u'~']


In [23]:
PUNCTUATION_TABLE = [
    # start,  stop
    (0x2000, 0x206f),  # General Punctuation
    (0x3000, 0x303f),  # CJK Symbols and Punctuation
    (0xff00, 0xffef),  # Halfwidth and Fullwidth Forms
    (0x00, 0x2f),  # ascii special charactors
    # (0x30, 0x39), # number 0~9
    (0x3a, 0x40),  # ascii special charactors
    (0x5b, 0x60),  # ascii special charactors
    (0x7b, 0xff),  # ascii special charactors
]
PUNCTUATION_RANGE = reduce(lambda x, y: x.union(y), [
                           range(start, stop + 1) for start, stop in PUNCTUATION_TABLE], set())



def isPUNC(uchar):
    if ord(uchar) in PUNCTUATION_RANGE:
        return True
    else:
        return False

In [25]:
def FindTok(cutlist, char):
    if char in cutlist:
        return True
    else:
        return False
     
def Cut(cutlist,lines):   
    l = []   
    line = []   
        
    for i in lines:   
        if FindTok(cutlist,i):
            line.append(i)
            l.append(''.join(line))
            #l.append(i)
            line = []   
        else:   
            line.append(i)   
    return l  
#from nltk.tokenize.stanford_segmenter import StanfordSegmenter
cutlist = "[。，,！……!《》<>\"':：？/?、/|“”‘’；]{}（）{}【】()｛｝（）：？！。，;、~——+％%`:“”＂'‘\n\r".decode('utf-8') 
for lines in file("testfile.txt"):  
    l = Cut(list(cutlist),list(lines.decode('utf-8')))  
    for line in l:  
       if line.strip() <> "":#这里可能包含空格  
            li = line.strip().split()  
            for sentence in li:  
                print "se:",sentence  

se: 漢皇重色思傾國，
se: 御宇多年求不得。
se: 楊家有女初長成，
se: 養在深閨人未識。
se: 天生麗質難自棄，
se: 一朝選在君王側。
se: 回眸一笑百媚生，
se: 六宮粉黛無顏色。
se: 春寒賜浴華清池，
se: 溫泉水滑洗凝脂。
se: 侍兒扶起嬌無力，
se: 始是新承恩澤時。
se: 雲鬢花顏金步搖，
se: 芙蓉帳暖度春宵。
se: 春宵苦短日高起，
se: 從此君王不早朝。
se: 承歡侍宴無閒暇，
se: 春從春遊夜專夜。
se: 後宮佳麗三千人，
se: 三千寵愛在一身。
se: 金屋妝成嬌侍夜，
se: 玉樓宴罷醉和春。
se: 姊妹兄弟皆列土，
se: 可憐光彩生門戶。
se: 遂令天下父母心，
se: 不重生男重生女。
se: 驪宮高處入青雲，
se: 仙樂風飄處處聞。
se: 緩歌慢舞凝絲竹，
se: 盡日君王看不足。
se: 漁陽鼙鼓動地來，
se: 驚破霓裳羽衣曲。
se: 九重城闕煙塵生，
se: 千乘萬騎西南行。
se: 翠華颻颻行復止，
se: 西出都門百餘里。
se: 六軍不發無奈何，
se: 宛轉蛾眉馬前死。
se: 花鈿委地無人收，
se: 翠翹金雀玉搔頭。
se: 君王掩面救不得，
se: 回看血淚相和流。
se: 黃埃散漫風蕭索，
se: 雲棧縈紆登劍閣。
se: 峨嵋山下少人行，
se: 旌旗無光日色薄。
se: 蜀江水碧蜀山青，
se: 聖主朝朝暮暮情。
se: 行宮見月傷心色，
se: 夜雨聞鈴腸斷聲。
se: 天旋日轉回龍馭，
se: 到此躊躇不能去。
se: 馬嵬坡下泥土中，
se: 不見玉顏空死處。
se: 君臣相顧盡霑衣，
se: 東望都門信馬歸。
se: 歸來池苑皆依舊，
se: 太液芙蓉未央柳。
se: 芙蓉如面柳如眉，
se: 對此如何不淚垂。
se: 春風桃李花開日，
se: 秋雨梧桐葉落時。
se: 西宮南內多秋草，
se: 落葉滿階紅不掃。
se: 梨園弟子白髮新，
se: 椒房阿監青娥老。
se: 夕殿螢飛思悄然，
se: 孤燈挑盡未成眠。
se: 遲遲鐘鼓初長夜，
se: 耿耿星河欲曙天。
se: 鴛鴦瓦冷霜華重，
se: 翡翠衾寒誰與共。
se: 悠悠生死別經年，
se: 魂魄不曾來入夢。
se: 臨邛道士鴻都客，
se: 能以精誠致魂魄。
se: 為感君王輾轉思，

In [26]:
def cut_sentence_new(words):
    # words = (words).decode('utf8')
    start = 0
    i = 0
    sents = []
    cutlist = "[。，,！……!《》<>\"':：？/?、/|“”‘’；]{}（）{}【】()｛｝（）：？！。，;、~——+％%`:“”＂'‘\n\r".decode('utf-8')
    punt_list = cutlist
    for word in words:
        if word in punt_list and token not in punt_list: #检查标点符号下一个字符是否还是标点
            sents.append(words[start:i+1])
            start = i+1
            i += 1
        else:
            i += 1
            token = list(words[start:i+2]).pop() # 取下一个字符
    if start < len(words):
        sents.append(words[start:])
    return sents
f = open("testfile.txt",'rb')
for s in cut_sentence_new(f.read().decode('utf-8')):
    print 'se:',s.strip()


se: 漢皇重色思傾國，
se: 御宇多年求不得。
se: 楊家有女初長成，
se: 養在深閨人未識。
se: 天生麗質難自棄，
se: 一朝選在君王側。
se: 回眸一笑百媚生，
se: 六宮粉黛無顏色。
se: 春寒賜浴華清池，
se: 溫泉水滑洗凝脂。
se: 侍兒扶起嬌無力，
se: 始是新承恩澤時。
se: 雲鬢花顏金步搖，
se: 芙蓉帳暖度春宵。
se: 春宵苦短日高起，
se: 從此君王不早朝。
se: 承歡侍宴無閒暇，
se: 春從春遊夜專夜。
se: 後宮佳麗三千人，
se: 三千寵愛在一身。
se: 金屋妝成嬌侍夜，
se: 玉樓宴罷醉和春。
se: 姊妹兄弟皆列土，
se: 可憐光彩生門戶。
se: 遂令天下父母心，
se: 不重生男重生女。
se: 驪宮高處入青雲，
se: 仙樂風飄處處聞。
se: 緩歌慢舞凝絲竹，
se: 盡日君王看不足。
se: 漁陽鼙鼓動地來，
se: 驚破霓裳羽衣曲。
se: 九重城闕煙塵生，
se: 千乘萬騎西南行。
se: 翠華颻颻行復止，
se: 西出都門百餘里。
se: 六軍不發無奈何，
se: 宛轉蛾眉馬前死。
se: 花鈿委地無人收，
se: 翠翹金雀玉搔頭。
se: 君王掩面救不得，
se: 回看血淚相和流。
se: 黃埃散漫風蕭索，
se: 雲棧縈紆登劍閣。
se: 峨嵋山下少人行，
se: 旌旗無光日色薄。
se: 蜀江水碧蜀山青，
se: 聖主朝朝暮暮情。
se: 行宮見月傷心色，
se: 夜雨聞鈴腸斷聲。
se: 天旋日轉回龍馭，
se: 到此躊躇不能去。
se: 馬嵬坡下泥土中，
se: 不見玉顏空死處。
se: 君臣相顧盡霑衣，
se: 東望都門信馬歸。
se: 歸來池苑皆依舊，
se: 太液芙蓉未央柳。
se: 芙蓉如面柳如眉，
se: 對此如何不淚垂。
se: 春風桃李花開日，
se: 秋雨梧桐葉落時。
se: 西宮南內多秋草，
se: 落葉滿階紅不掃。
se: 梨園弟子白髮新，
se: 椒房阿監青娥老。
se: 夕殿螢飛思悄然，
se: 孤燈挑盡未成眠。
se: 遲遲鐘鼓初長夜，
se: 耿耿星河欲曙天。
se: 鴛鴦瓦冷霜華重，
se: 翡翠衾寒誰與共。
se: 悠悠生死別經年，
se: 魂魄不曾來入夢。
se: 臨邛道士鴻都客，
se: 能以精誠致魂魄。
se: 為感君王輾轉思，

In [38]:
from nltk.tokenize.stanford_segmenter import StanfordSegmenter
segmenter = StanfordSegmenter(path_to_jar='./stanford-segmenter/stanford-segmenter-3.4.1.jar', path_to_sihan_corpora_dict='./stanford-segmenter/data', path_to_model='./stanford-segmenter/data/pku.gz', path_to_dict='./stanford-segmenter/data/dict-chris6.ser.gz')
dir(segmenter)
for s in segmenter.tokenize_sents(f.read().decode('utf-8')):
    print s

In [46]:
import nltk.data

tokenizer = nltk.data.load('tokenizers/punkt/chinese.pickle')
fp = open("testfile.txt")
data = fp.read().decode('utf-8')
print '\n-----\n'.join(tokenizer.tokenize(data))

漢皇重色思傾國，	　御宇多年求不得。
楊家有女初長成，	　養在深閨人未識。
天生麗質難自棄，	　一朝選在君王側。
回眸一笑百媚生，	　六宮粉黛無顏色。
春寒賜浴華清池，	　溫泉水滑洗凝脂。
侍兒扶起嬌無力，	　始是新承恩澤時。
雲鬢花顏金步搖，	　芙蓉帳暖度春宵。
春宵苦短日高起，	　從此君王不早朝。
承歡侍宴無閒暇，	　春從春遊夜專夜。
後宮佳麗三千人，	　三千寵愛在一身。
金屋妝成嬌侍夜，	　玉樓宴罷醉和春。
姊妹兄弟皆列土，	　可憐光彩生門戶。
遂令天下父母心，	　不重生男重生女。
驪宮高處入青雲，	　仙樂風飄處處聞。
緩歌慢舞凝絲竹，	　盡日君王看不足。
漁陽鼙鼓動地來，	　驚破霓裳羽衣曲。
九重城闕煙塵生，	　千乘萬騎西南行。
翠華颻颻行復止，	　西出都門百餘里。
六軍不發無奈何，	　宛轉蛾眉馬前死。
花鈿委地無人收，	　翠翹金雀玉搔頭。
君王掩面救不得，	　回看血淚相和流。
黃埃散漫風蕭索，	　雲棧縈紆登劍閣。
峨嵋山下少人行，	　旌旗無光日色薄。
蜀江水碧蜀山青，	　聖主朝朝暮暮情。
行宮見月傷心色，	　夜雨聞鈴腸斷聲。
天旋日轉回龍馭，	　到此躊躇不能去。
馬嵬坡下泥土中，	　不見玉顏空死處。
君臣相顧盡霑衣，	　東望都門信馬歸。
歸來池苑皆依舊，	　太液芙蓉未央柳。
芙蓉如面柳如眉，	　對此如何不淚垂。
春風桃李花開日，	　秋雨梧桐葉落時。
西宮南內多秋草，	　落葉滿階紅不掃。
梨園弟子白髮新，	　椒房阿監青娥老。
夕殿螢飛思悄然，	　孤燈挑盡未成眠。
遲遲鐘鼓初長夜，	　耿耿星河欲曙天。
鴛鴦瓦冷霜華重，	　翡翠衾寒誰與共。
悠悠生死別經年，	　魂魄不曾來入夢。
臨邛道士鴻都客，	　能以精誠致魂魄。
為感君王輾轉思，	　遂教方士殷勤覓。
排雲馭氣奔如電，	　昇天入地求之遍。
上窮碧落下黃泉，	　兩處茫茫皆不見。
忽聞海上有仙山，	　山在虛無縹緲間。
樓閣玲瓏五雲起，	　其中綽約多仙子。
中有一人字太真，	　雪膚花貌參差是。
金闕西廂叩玉扃，	　轉教小玉報雙成。
聞道漢家天子使，	　九華帳裏夢魂驚。
攬衣推枕起徘徊，	　珠箔銀屏迤邐開。
雲髻半偏新睡覺，	　花冠不整下堂來。
風吹仙袂飄颻舉，	　猶似霓裳羽衣舞。
玉容寂寞淚闌干，	　梨花一枝春帶雨。
含情凝睇謝君王，	　一別音容兩渺茫。
昭陽殿裏恩愛絕，	　蓬萊宮中日月長。
回頭下望人寰處，	　不見

In [84]:
import codecs
#處理編碼的套件
import operator
##處理字典檔排序的套件
cut_list = "<>/:：;；,、＂’，.。！？｢\"\'\\\n\r《》“”!@#$%^&*()".decode("utf-8")  ##列出標點符號，並轉換成utf-8的格式 

def cutSentence(text_path, keywords): ##放入原始文章路徑, 增加斷詞的list
    text = codecs.open(text_path,"r","utf-8")   #開檔
    sent = ""
    textList = []
       
    for line in text.readlines():
        line = line.strip() ##清除空白
        
        for keyword in keywords:  #清除關鍵字
            line = "".join(line.split(keyword))
            
        for word in line:
            if word not in sep_list: #如果文字不是標點符號，就把字加到句子中
                sent += word
                #print sentence
            else:
                sentence = sent.strip()
                if len(sentence) != 0:
                    textList.append(sentence.strip()) #如果遇到標點符號，把句子加到 text list中
                sent = ""
                #print textList

    return textList#傳回一個文字陣列

for s in cutSentence("testfile.txt", ""):
    print 'se:',s


se: 漢皇重色思傾國
se: 御宇多年求不得
se: 楊家有女初長成
se: 養在深閨人未識
se: 天生麗質難自棄
se: 一朝選在君王側
se: 回眸一笑百媚生
se: 六宮粉黛無顏色
se: 春寒賜浴華清池
se: 溫泉水滑洗凝脂
se: 侍兒扶起嬌無力
se: 始是新承恩澤時
se: 雲鬢花顏金步搖
se: 芙蓉帳暖度春宵
se: 春宵苦短日高起
se: 從此君王不早朝
se: 承歡侍宴無閒暇
se: 春從春遊夜專夜
se: 後宮佳麗三千人
se: 三千寵愛在一身
se: 金屋妝成嬌侍夜
se: 玉樓宴罷醉和春
se: 姊妹兄弟皆列土
se: 可憐光彩生門戶
se: 遂令天下父母心
se: 不重生男重生女
se: 驪宮高處入青雲
se: 仙樂風飄處處聞
se: 緩歌慢舞凝絲竹
se: 盡日君王看不足
se: 漁陽鼙鼓動地來
se: 驚破霓裳羽衣曲
se: 九重城闕煙塵生
se: 千乘萬騎西南行
se: 翠華颻颻行復止
se: 西出都門百餘里
se: 六軍不發無奈何
se: 宛轉蛾眉馬前死
se: 花鈿委地無人收
se: 翠翹金雀玉搔頭
se: 君王掩面救不得
se: 回看血淚相和流
se: 黃埃散漫風蕭索
se: 雲棧縈紆登劍閣
se: 峨嵋山下少人行
se: 旌旗無光日色薄
se: 蜀江水碧蜀山青
se: 聖主朝朝暮暮情
se: 行宮見月傷心色
se: 夜雨聞鈴腸斷聲
se: 天旋日轉回龍馭
se: 到此躊躇不能去
se: 馬嵬坡下泥土中
se: 不見玉顏空死處
se: 君臣相顧盡霑衣
se: 東望都門信馬歸
se: 歸來池苑皆依舊
se: 太液芙蓉未央柳
se: 芙蓉如面柳如眉
se: 對此如何不淚垂
se: 春風桃李花開日
se: 秋雨梧桐葉落時
se: 西宮南內多秋草
se: 落葉滿階紅不掃
se: 梨園弟子白髮新
se: 椒房阿監青娥老
se: 夕殿螢飛思悄然
se: 孤燈挑盡未成眠
se: 遲遲鐘鼓初長夜
se: 耿耿星河欲曙天
se: 鴛鴦瓦冷霜華重
se: 翡翠衾寒誰與共
se: 悠悠生死別經年
se: 魂魄不曾來入夢
se: 臨邛道士鴻都客
se: 能以精誠致魂魄
se: 為感君王輾轉思
se: 遂教方士殷勤覓
se: 排雲馭氣奔如電
se: 昇天入地求之遍
se: 上窮碧落下黃泉
se: 兩處茫茫皆不見
se: 忽聞海上有仙山
se: 